 Seattle - Airbnb Data Set.



In order to get a better grip over the Airbnb data set. Lets wrangle to get a better understanding, and prepare the data set.


In [13]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from IPython import display
%matplotlib inline



In [14]:
airbnb_listing_df = pd.read_csv('D:\OneDrive\Projects\predict_airbnb_prices_seattle\Predict-Airbnb-Prices-in-Seattle\Data\Airbnb_Seattle\seattle_airbnb_listing.csv')

airbnb_listing_df = pd.read_csv('Data/Airbnb_Seattle/seattle_airbnb_neighbourhoods.csv')

In [3]:
airbnb_df.shape

In [4]:
airbnb_df.head()

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365
0,2318,"Casa Madrona - Urban Oasis, 1 block from the P...",2536,Megan,Central Area,Madrona,47.610819,-122.290816,Entire home/apt,296,4,20,9/18/2018,0.16,3,345
1,4291,Sunrise in Seattle Master Suite,35749,Jess & Joey,Other neighborhoods,Roosevelt,47.687801,-122.313427,Private room,82,2,63,10/6/2018,0.96,5,365
2,5682,"Cozy Studio, min. to downtown -WiFi",8993,Maddy,Delridge,South Delridge,47.523979,-122.359891,Entire home/apt,48,3,461,11/2/2018,4.37,1,24
3,6606,"Fab, private seattle urban cottage!",14942,Joyce,Other neighborhoods,Wallingford,47.654109,-122.337605,Entire home/apt,90,2,134,10/21/2018,1.18,3,287
4,9419,Glorious sun room w/ memory foambed,30559,Angielena,Other neighborhoods,Georgetown,47.550620,-122.320135,Private room,65,2,130,9/9/2018,1.29,5,336


In [5]:
airbnb_df.iloc[:10,:]

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365
0,2318,"Casa Madrona - Urban Oasis, 1 block from the P...",2536,Megan,Central Area,Madrona,47.610819,-122.290816,Entire home/apt,296,4,20,9/18/2018,0.16,3,345
1,4291,Sunrise in Seattle Master Suite,35749,Jess & Joey,Other neighborhoods,Roosevelt,47.687801,-122.313427,Private room,82,2,63,10/6/2018,0.96,5,365
2,5682,"Cozy Studio, min. to downtown -WiFi",8993,Maddy,Delridge,South Delridge,47.523979,-122.359891,Entire home/apt,48,3,461,11/2/2018,4.37,1,24
3,6606,"Fab, private seattle urban cottage!",14942,Joyce,Other neighborhoods,Wallingford,47.654109,-122.337605,Entire home/apt,90,2,134,10/21/2018,1.18,3,287
4,9419,Glorious sun room w/ memory foambed,30559,Angielena,Other neighborhoods,Georgetown,47.550620,-122.320135,Private room,65,2,130,9/9/2018,1.29,5,336
5,9460,Downtown/Convention Ctr B&B - Nice!,30832,Siena,Downtown,First Hill,47.608010,-122.328739,Private room,80,3,399,10/28/2018,3.56,4,133
6,9531,The Adorable Sweet Orange Craftsman,31481,Cassie,West Seattle,Fairmount Park,47.555389,-122.384739,Entire home/apt,165,2,35,10/8/2018,0.42,2,267
7,9534,The Coolest Tangerine Dream MIL!,31481,Cassie,West Seattle,Fairmount Park,47.556236,-122.385980,Entire home/apt,125,2,36,10/29/2018,0.43,2,352
8,9596,"the down home , spacious, central and fab!",14942,Joyce,Other neighborhoods,Wallingford,47.654787,-122.336520,Entire home/apt,120,2,76,10/1/2018,0.84,3,94
9,9909,Luna Park Lower - West Seattle,33360,Laura,West Seattle,Fairmount Park,47.565209,-122.373748,Entire home/apt,125,3,57,10/23/2018,3.75,8,22
